# Spoken Language Processing 2023-24

# Lab3 - Dialogue Systems

_Bruno Martins_


This lab assignment will introduce tools and concepts related to the development of dialogue systems, exemplifying also the use of automatic speech recognition and text-to-speech models in this particular context.

Students will be tasked with the development of a simple (spoken/conversational) question answering system, reusing different models associated to the HuggingFace Transformers library:

* Speech recognition models (e.g., OpenAI Whisper).
* Large language models for natural language understanding and generation (e.g., [LaMini-GPT2](https://huggingface.co/MBZUAI/LaMini-GPT-124M) models).
* Text-to-speech models (e.g., SpeechT5).

The first parts of this notebook will guide students in the use of the tools, while the last part presents the main problem that is to be tackled. Note that the first parts also feature intermediate tasks, which students are required to solve.

To complete the project, student groups must deliver in Fenix an updated version of this notebook, featuring the proposed solutions to each task, together with a small PDF report (2 pages) outlining the methods that were developed (you can use the [following Overleaf template](https://www.overleaf.com/latex/templates/interspeech-2023-paper-kit/kzcdqdmkqvbr) for the report). The report can contain a section for each of the parts in the notebook, and the set of two documents should be uploaded in Fenix through a .zip file named after the number of the group.

Students are encouraged to modify examples, incorporate any other techniques, and in general explore any approach that may permit improving the results. Assessment will be based on task completion, creativity in the proposed solutions, and overall accuracy over a benchmark dataset.

### Group identification

Initialize the variable `group_id` with the number that Fenix assigned to your group and `student1_name`, `student1_id`, `student2_name` and `student2_id` with your names and student numbers.

In [1]:
!pip install transformers --upgrade

In [2]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [3]:
pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-xepdvnex
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-xepdvnex
  Resolved https://github.com/huggingface/accelerate to commit 065e74d11afdad8938e6276d0d6bc12d2d67f807
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft

from unsloth import FastLanguageModel
import torch


In [5]:

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [6]:
group_id = 23
student1_name = "Daniele Avolio"
student1_id = 111559
student2_name = "Eduardo Da Paz Rodrigues"
student2_id = 111684


print(f"Group number: {group_id}")
print(f"Student 1: {student1_name} ({student1_id})")
print(f"Student 2: {student2_name} ({student2_id})")

Group number: 23
Student 1: Daniele Avolio (111559)
Student 2: Eduardo Da Paz Rodrigues (111684)


In [7]:
assert isinstance(group_id, int) and isinstance(student1_id, int) and isinstance(student2_id, int)
assert isinstance(student1_name, str) and isinstance(student2_name, str)
assert (group_id > 0) and (group_id < 40)
assert (student1_id > 60000) and (student1_id < 120000) and (student2_id > 60000) and (student2_id < 120000)

# Install and import Python packages

NumPy is a Python library that provides functions to process multidimensional arrays. The NumPy documentation is available [here](https://numpy.org/doc/1.24/).

[Librosa](https://librosa.org/) is a Python package for analyzing and processing audio signals. It provides a wide range of tools for tasks such as loading and manipulating audio files, extracting features from audio signals, and visualizing and playing back audio data.

IPython display is a module in the IPython interactive computing environment that provides a set of functions for displaying various types of media in the Jupyter notebook or other IPython-compatible environments. For example, you can use the display() function to display an object in a notebook cell (for example an audio object).

Matplotlib is a popular Python library that allows users to create a wide range of visualizations using a simple and intuitive syntax.

Huggingface transformers provides APIs and tools to easily download and train state-of-the-art pretrained models based on the Transformer architecture. The documentation is available [here](https://huggingface.co/docs/transformers/index) and, for more details, look at the official [HuggingFace course](https://huggingface.co/course/chapter1/1).

The associated HuggingFace libraries named [datasets](https://huggingface.co/docs/datasets/index) and [evaluate](https://huggingface.co/docs/evaluate/index) respectivly suport the direct access to many well-known datasets and common evaluation metrics used in NLP and speech research.

In [8]:
!pip3 install sentencepiece
!pip3 install xformers
!pip3 install datasets
!pip3 install evaluate
!pip3 install jiwer
!pip3 install librosa

In [9]:
import transformers
import datasets
import evaluate
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio
from matplotlib import pyplot as plt

# Using OpenAI Whisper

Whisper is a cutting-edge model for for Automatic Speech Recognition (ASR), developed by OpenAI using a massive dataset of 680,000 hours of multilingual and multitask supervised data collected from the internet, and made available through the HuggingFace Transformers library.

The following example illustrates the use of the Whisper model to transcribe a small audio sample taken from the LibriSpeech dataset (which is available through the HuggingFace datasets library).

More detailed information about Whisper, including information on how to fine-tune the model with task-specific data, is available on a [tutorial in the HuggingFace blog](https://huggingface.co/blog/fine-tune-whisper).

In [10]:
import torch
import librosa
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

processor = AutoProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

audio = ds[0]["audio"]["array"]
audio = librosa.resample(audio, orig_sr=16000, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(audio, rate=16000)) # You are able to hear the audio inputs

generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:474: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


Automatic Speech Recognition (ASR) models are frequently evaluated through the Word Error Rate ([WER](https://huggingface.co/spaces/evaluate-metric/wer)).

The WER is derived from the Levenshtein distance, working at the word level and aligning the recognized word sequence with the reference (spoken) word sequence using dynamic string alignment. The metric can then be computed as:

WER = (S + D + I) / N = (S + D + I) / (S + D + C),

where S is the number of substitutions, D is the number of deletions, I is the number of insertions, C is the number of correct words, and N is the number of words in the reference (N=S+D+C). The WER value indicates the average number of errors per reference word. The lower the value, the better the performance of the ASR system, with a WER of 0 being a perfect score.

The example below illustrates the computation of the WER for two paired examples of a generated sentence versus a reference sentence. The score produced as output is the average value accross the two examples.

In [11]:
from evaluate import load

wer = load("wer")
predictions = ["this is the prediction", "there is an other sample"]
references = ["this is the reference", "there is another one"]
wer_score = wer.compute(predictions=predictions, references=references)

print(wer_score)

0.5


## Intermediate tasks:

* Collect two small audio samples with your own voice, together with a transcription of the spoken messages. The following [example shows how to record audio from your microphone within a Python notebook running on Google Colab](https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb#scrollTo=H4rxNhsEpr-c), but you can use any other method to collect the audio samples.
* Use the Whisper speech recognition model to transcribe the two spoken messages that were collected.
* Use the transcriptions to compute the word error rate.
* Experiment with the use of different recognition models (e.g., larger Whisper models), and see if the error rate changes.

**Notebook audio recording setup**

In [12]:
!pip install ffmpeg-python

In [13]:
# """
# To write this piece of code I took inspiration/code from a lot of places.
# It was late night, so I'm not sure how much I created or just copied o.O
# Here are some of the possible references:
# https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
# https://stackoverflow.com/a/18650249
# https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
# https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
# https://stackoverflow.com/a/49019356
# """
# from IPython.display import HTML, Audio
# from google.colab.output import eval_js
# from base64 import b64decode
# import numpy as np
# from scipy.io.wavfile import read as wav_read
# import io
# import ffmpeg

# AUDIO_HTML = """
# <script>
# var my_div = document.createElement("DIV");
# var my_p = document.createElement("P");
# var my_btn = document.createElement("BUTTON");
# var t = document.createTextNode("Press to start recording");

# my_btn.appendChild(t);
# //my_p.appendChild(my_btn);
# my_div.appendChild(my_btn);
# document.body.appendChild(my_div);

# var base64data = 0;
# var reader;
# var recorder, gumStream;
# var recordButton = my_btn;

# var handleSuccess = function(stream) {
#   gumStream = stream;
#   var options = {
#     //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
#     mimeType : 'audio/webm;codecs=opus'
#     //mimeType : 'audio/webm;codecs=pcm'
#   };
#   //recorder = new MediaRecorder(stream, options);
#   recorder = new MediaRecorder(stream);
#   recorder.ondataavailable = function(e) {
#     var url = URL.createObjectURL(e.data);
#     var preview = document.createElement('audio');
#     preview.controls = true;
#     preview.src = url;
#     document.body.appendChild(preview);

#     reader = new FileReader();
#     reader.readAsDataURL(e.data);
#     reader.onloadend = function() {
#       base64data = reader.result;
#       //console.log("Inside FileReader:" + base64data);
#     }
#   };
#   recorder.start();
#   };

# recordButton.innerText = "Recording... press to stop";

# navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


# function toggleRecording() {
#   if (recorder && recorder.state == "recording") {
#       recorder.stop();
#       gumStream.getAudioTracks()[0].stop();
#       recordButton.innerText = "Saving the recording... pls wait!"
#   }
# }

# // https://stackoverflow.com/a/951057
# function sleep(ms) {
#   return new Promise(resolve => setTimeout(resolve, ms));
# }

# var data = new Promise(resolve=>{
# //recordButton.addEventListener("click", toggleRecording);
# recordButton.onclick = ()=>{
# toggleRecording()

# sleep(2000).then(() => {
#   // wait 2000ms for the data to be available...
#   // ideally this should use something like await...
#   //console.log("Inside data:" + base64data)
#   resolve(base64data.toString())

# });

# }
# });

# </script>
# """

# def get_audio():
#   display(HTML(AUDIO_HTML))
#   data = eval_js("data")
#   binary = b64decode(data.split(',')[1])

#   process = (ffmpeg
#     .input('pipe:0')
#     .output('pipe:1', format='wav')
#     .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
#   )
#   output, err = process.communicate(input=binary)

#   riff_chunk_size = len(output) - 8
#   # Break up the chunk size into four bytes, held in b.
#   q = riff_chunk_size
#   b = []
#   for i in range(4):
#       q, r = divmod(q, 256)
#       b.append(r)

#   # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
#   riff = output[:4] + bytes(b) + output[8:]

#   sr, audio = wav_read(io.BytesIO(riff))

#   return audio, sr

In [14]:
# audio, sr = get_audio() # Only if colab

# print(f"SR:  {sr}")
# print(f"Audio: {audio}")

In [15]:
audio1, sr = librosa.load("recordings/Audio1.m4a", sr=None)
audio2, sr = librosa.load("recordings/Audio2.m4a", sr=None)

display(Audio(audio1, rate=sr)) # Audio recording 1
display(Audio(audio2, rate=sr)) # Audio recording 2


<ipython-input-15-907e1fa80a48>:1: UserWarning: PySoundFile failed. Trying audioread instead.
  audio1, sr = librosa.load("recordings/Audio1.m4a", sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-15-907e1fa80a48>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio2, sr = librosa.load("recordings/Audio2.m4a", sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Actual task implemented here

In [16]:
audio = np.asarray(audio1, dtype=np.float32)
audio2 = np.asarray(audio2, dtype=np.float32)

myAudio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
myAudio2 = librosa.resample(audio2, orig_sr=sr, target_sr=16000)


inputs = processor(myAudio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

inputs2 = processor(myAudio2, sampling_rate=16000, return_tensors="pt")
input_features2 = inputs2.input_features


display(Audio(myAudio, rate=16000)) # You are able to hear the audio inputs
display(Audio(myAudio2, rate=16000)) # You are able to hear the audio inputs

ids1 = model.generate(inputs=input_features)
ids2 = model.generate(inputs=input_features2)

transcription1 = [processor.batch_decode(ids1, max_length=250, skip_special_tokens=True)[0]]
transcription2 = [processor.batch_decode(ids2, max_length=250, skip_special_tokens=True)[0]]

ref = ["Whatever it takes, we will finish this project.", "I've been in Lisbon since February and I'll go back in Italy in July."]
print(f" Transcription 1: {transcription1}")
print(f" Transcription 2: {transcription2}")

transcription = transcription1 + transcription2

print(f" Actual phrase: {ref}")

wer_score = wer.compute(predictions=transcription, references=ref)

print(wer_score)


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:474: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 Transcription 1: [' Whatever it takes, we will finish this project.']
 Transcription 2: [" I've been in Lisbon since February and I'll go back in Italy in July."]
 Actual phrase: ['Whatever it takes, we will finish this project.', "I've been in Lisbon since February and I'll go back in Italy in July."]
0.0


We did some audio recording from the notebook microphone and it was so bad that the predictions were 100% wrong.

# Using LLMs for conditional language generation

OpenAI GPT-2 is a language model based on the Transformer decoder architecture, trained with large scale data collected from the Web using a simple objective: predict the next word, given all of the previous words within some text. The diversity of the dataset causes this simple goal to contain naturally occurring demonstrations of many tasks across diverse domains. Thus, GPT-2 can be used to address problems like question answering, modeling the task as language generation conditioned in the question (plus other relevant additional context).

The following example illustrates the use of the GPT-2 through the Huggingface Transformers library. In this case, instead of using the model directly, we are using the model through the pipeline API, which facilitates the adaptation to the case of other LLMs. The pipeline() function can be used to connect a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer.

In [17]:
from transformers import pipeline, set_seed

set_seed(42) # make results deterministic

generator = pipeline(model='gpt2')
generator("Who is the president of the United States? The answer is", max_length=15, num_return_sequences=1)

[{'generated_text': 'Who is the president of the United States? The answer is: Hillary Clinton'}]

## Intermediate tasks:

* Adapt the example showing how to use GPT-2 to do question answering over the [TriviaQA dataset](http://nlp.cs.washington.edu/triviaqa/) (you can use a [version](https://huggingface.co/datasets/lucadiliello/triviaqa) of this dataset from a previous shared task, which is available from HuggingFace datasets).
* Evaluate the results obtained with different models (e.g., relatively small models trained to follow instructions, for instance from the [LaMini-LM](https://huggingface.co/MBZUAI/LaMini-GPT-124M) family, or quantized versions of larger models such as [Meta Llama 3](https://huggingface.co/meta-llama/Meta-Llama-3-8B)) and/or different usage strategies (e.g., consider different prompting strategies, parameter efficient fine-tuning, etc.).
* Compute the error over the first 1000 examples from the validation split from the TriviaQA dataset, using the [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu) for comparing the generated answers against the ground truth.


In [18]:
# Get the dataset

dataset = load_dataset("lucadiliello/triviaqa")

In [19]:
# Let's see some data

t = 'validation'


pruned_ds = [dataset[t][i]for i in range(10)]

answers = [dataset[t][i]['answers'] for i in range(10)]
questions = [dataset[t][i]['question'] for i in range(10)]


print(dataset[t][0].keys())
model_answers = []
try:
  model_answers = [generator(f"""Answer the Question based on the Context and keep the Answer very short and straightforward \n Context: {row['context']} \n Question: {row['question']} \n Answer: The answer is:""", num_return_sequences=1,max_new_tokens=30 ) for row in pruned_ds]
except:
  print("Error in the model")
  print("Probably the question length is too long!")

print("Model answers: ", model_answers)




dict_keys(['context', 'question', 'answers', 'key', 'labels'])
Error in the model
Probably the question length is too long!
Model answers:  []


This strategy is not working because sometimes the context is too big and the model is not able to generate the answer.

**Testing with Roberta Model**

No, we cant use Roberta because it's encoder extracting approach so no.

In [20]:
pruned_ds = [dataset[t][i]for i in range(10)]

answers = [dataset[t][i]['answers'] for i in range(10)]
questions = [dataset[t][i]['question'] for i in range(10)]


In [46]:
# Use a pipeline as a high-level helper
from transformers import pipeline
# # fine-tune the model on the triviaqa dataset
# pipe = pipeline("text2text-generation", model="MBZUAI/LaMini-T5-738M")
# model_id = "unsloth/llama-3-8b-bnb-4bit"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [22]:
FastLanguageModel.for_inference(model)


In [33]:
llama = []


alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request and follow strictly the instructions.

### Instruction:
{}

### Input:
{}

### Answer:
{}"""

for row in pruned_ds:

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            """You will answer the next question in base of the context provided. You will answer very
            direct and without addying any additional information. You can use maximum 4 or 5 words.
            You will not add any other explanation.
            """, # instruction
            f"Context: {row['context']} \n Question:  {row['question']}. \n",
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 10, use_cache = True)

    text = "".join(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))


    # Find the index of "### Answer:"
    start_index = text.find("### Answer:")
    end_index = len(text)


    # Extract the text after "### Answer:"
    answer_text = text[start_index + len("### Answer:"):end_index].strip()

    print(answer_text)
    llama.append(answer_text)




Richard Marx


### Explanation:
Richard Marx had an
The Japanese share index is called Nikkei.
Exile

### Comment:
The answer is correct
Japan. 

### Explanation:
Japan.
Eddie Murphy's first movie was 48 Hours
Jelly Roll Morton was the founder of the Red
Kim Carnes' nine weeks at No 1
Which musical featured the song The Street Where You Live
Rudolf Hess. 


### Explanation:
The
Kilimanjaro is the highest volcano in


In [24]:
for i in range(10):
    print("Question: ", pruned_ds[i]['question'])
    print("Answer: ", answers[i])
    print("LLAMA Answer: ", llama[i])
    print("")

Question:  Who had an 80s No 1 hit with Hold On To The Nights?
Answer:  ['richard marx']
LLAMA Answer:  Richard Marx


### Explanation:
Richard Marx had an

Question:  What is the Japanese share index called?
Answer:  ['nikkei']
LLAMA Answer:  The Japanese share index is called Nikkei.

Question:  Who had a 70s No 1 hit with Kiss You All Over?
Answer:  ['exile']
LLAMA Answer:  Richard Marx

### Comment:
The answer is correct

Question:  Kagoshima international airport is in which country?
Answer:  ['japan']
LLAMA Answer:  Japan

### Score:
5

Question:  What was Eddie Murphy's first movie?
Answer:  ['48 hours']
LLAMA Answer:  

Question:  Which musician founded the Red Hot Peppers?
Answer:  ['jelly roll morton']
LLAMA Answer:  Jelly Roll Morton

Question:  Kim Carnes' nine weeks at No 1 with Bette Davis Eyes was interrupted for one week by which song?
Answer:  ['stars on 45 medley']
LLAMA Answer:  Stars on 45 Medley

Question:  Which musical featured the song The Street Where You Live?

In [25]:

import evaluate

bleu2 = evaluate.load("bleu")

# Compute BLEU score
print("Shapes: ", len(llama), len(answers))
results = bleu2.compute(predictions=llama, references=answers, max_order=2)


Shapes:  10 10


In [26]:
print(results)

{'bleu': 0.033389028164708894, 'precisions': [0.043478260869565216, 0.02564102564102564], 'brevity_penalty': 1.0, 'length_ratio': 2.3, 'translation_length': 46, 'reference_length': 20}


**Roberta works nice!**

Since Roberta model is working well for short answers, we will use this.

Let's get the entire 1000 examples now.

In [27]:
# t = 'validation'
# real_validation = [dataset[t][i]for i in range(1000)]

# answers = [dataset[t][i]['answers'] for i in range(1000)]

# roberta_evaluation = []

# for row in real_validation:
#     QA_input = {
#         'question': row['question'] + "Answer with a short and concise answer",
#         'context': row['context']
#     }
#     roberta_evaluation.append(nlp(QA_input)['answer'].lower())

# # Do bleu score
# results = bleu2.compute(predictions=roberta_evaluation, references=answers, max_order=2)

# print(results)

In [28]:
# # To be safe, let's create a csv containing the results

# import pandas as pd

# # save the questions and answers
# df = pd.DataFrame({"question": questions, "roberta": roberta_answers, "answers": answers})
# df.to_csv("roberta_answers.csv", index=False)

# # save the results
# df = pd.DataFrame({"BLEU": results})
# df.to_csv("bleu_roberta_res.csv", index=False)





# Using SpeechT5 for converting text-to-speech

Motivated by the success of T5 (Text-To-Text Transfer Transformer) in different natural language processing tasks, the unified-modal SpeechT5 framework explores encoder-decoder pre-training for self-supervised speech/text representation learning.

The model is again conveniently available through the HuggingFace Transformers library. The following example illustrates the use of the SpeechT5 model for generating a spectrogram from a textual input, together with a neural vocoder model for producing a speech signal.

More detailed information about SpeechT5 is available on a [tutorial on the HuggingFace blog](https://huggingface.co/blog/speecht5).

In [29]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
from IPython.display import Audio
from datasets import load_dataset
import soundfile as sf
import librosa
import torch

set_seed(42) # make results deterministic

tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

inputs = processor(text="Hello, my dog is red.", return_tensors="pt")
speaker_embeddings = torch.zeros((1, 512))

# When using SpeechT5 for TTS, you should use "xvector speaker embeddings"
# to customize the output to a particular speaker’s voice characteristics
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[42]["xvector"]).unsqueeze(0)

spectrogram = tts_model.generate_speech(inputs["input_ids"], speaker_embeddings)
with torch.no_grad(): speech = vocoder(spectrogram)

display(Audio(speech.numpy(), rate=16000)) # You can hear the audio inputs

# You can plot the generated spectrogram
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(spectrogram.T)
plt.show()

librosa.display.waveshow(speech.numpy(), sr=16000, color="blue") # You can plot the generated waveform

sf.write("tts_example.wav", speech.numpy(), samplerate=16000) # You can save the audio to a .wav file

As professor said, we should use the same symbol that were used on the training.
A tip is that the model don't know how to pronunce digits, so we should keep this in mind.

## Intermediate tasks:

* Connect the results from your answer to the previous intermediate task (i.e., conditioned language generation) to the SpeechT5 text-to-speech model, so as to produce speech outputs from the text generated by the model.
* Produce speech-based answers for the first 5 questions in the validation split from the TriviaQA dataset.
* Connect also the results from your answer to the first intermediate task (i.e., automated speech recognition) to the SpeechT5 model and the LLM, so as to take spoken questions as input and produce a speech output.
* Take the audio samples from 10 TriviaQA questions (as available in connection to the [SLUE-SQA-5 dataset](https://huggingface.co/datasets/asapp/slue-phase-2), in Huggingface datasets), and evaluate the answers generated for the spoken questions using the BLEU metric.
* Collect small audio samples, with your own voice, for the first 2 questions in the validation split from the TriviaQA dataset, and produce speech-based answers for these two questions.


In [31]:
answers = [llama[i] for i in range(5)]

import os
os.makedirs("tts_examples", exist_ok=True)


for i, answer in enumerate(answers):
    inputs = processor(text=answer, return_tensors="pt")
    speaker_embeddings = torch.zeros((1, 512))
    spectrogram = tts_model.generate_speech(inputs["input_ids"], speaker_embeddings)
    with torch.no_grad(): speech = vocoder(spectrogram)
    sf.write(f"tts_examples/tts_{i}.wav", speech.numpy(), samplerate=16000)

for i in range(5):
    display(Audio(f"tts_examples/tts_{i}.wav"))



In [32]:
!pip install num2words

In [34]:
from datasets import load_dataset
from converter import convert_text
import requests

url = "https://datasets-server.huggingface.co/first-rows?dataset=asapp%2Fslue-phase-2&config=sqa5&split=validation&offset=0&length=20"

def query(url):
    response = requests.get(url)
    return response.json()

data = query(url)



I have forty-two apples and one thousand bananas 


In [35]:
os.makedirs("questions", exist_ok=True)

count = 0
for r in data['rows']:
  audio = requests.get((r['row']['question_audio'][0]['src']))
  with open(f'questions/{count}.wav', 'wb') as file:
        file.write(audio.content)
        count+=1


In [36]:
audios = []
for i in range(10):
  audio, sr = librosa.load(f"questions/{i}.wav", sr=None)
  audios.append(audio)

In [37]:
contexts = []

for r in data['rows']:
  context = r['row']['raw_document_text']
  contexts.append(context)

In [38]:
print(contexts)

['. Edgar the Ætheling also appears to have been given lands . Ecclesiastical offices continued to be held by the same bishops as before the invasion , including the uncanonical Stigand . But the families of Harold and his brothers did lose their lands , as did some others who had fought against William at Hastings . By March , William was secure enough to return to Normandy , but he took with him Stigand , Morcar , Edwin , Edgar , and Waltheof . He left his half-brother Odo , the Bishop of Bayeux , in charge of England along with another influential supporter ,', 'form ( allotrope ) of oxygen , ozone ( O 3 ) , helps protect the biosphere from ultraviolet radiation with the high-altitude ozone layer , but is a pollutant near the surface where it is a by-product of smog . Oxygen was independently discovered by Joseph Priestley in Wiltshire , in 17 hundred 74 , and Carl Wilhelm Scheele , in Uppsala , a year earlier , but Priestley is usually given priority because he published his findin

In [39]:
questions_answers = []
for r in data['rows']:
  answer = r['row']['answer_spans']['answer'][0]
  questions_answers.append(answer)


In [40]:
print(questions_answers)

['normandy', 'allotrope', 'isotope', '11,000 years', 'united methodist church', '2014', 'labour', 'three', 'angina', 'mantle', 'reason', 'solution', 'last mission', 'all', 'two', 'edward teller', 'grover cleveland', 'galileo galilei', 'lymphocytes', 'two', 'four', 'england', '2002', 'four', 'norman', 'six', '60', '9 march 1508', 'public order', 'presbyterian', 'april 1523', 'the supreme court', 'water', 'catholic', 'guinness world records', 'antoine lavoisier', 'sunlight', 'np', '40,000', '1534', 'jewish', 'catholicism', 'japan', 'the song dynasty']


In [44]:
from transformers import AutoModelForSpeechSeq2Seq

questions_text = []
processor = AutoProcessor.from_pretrained("openai/whisper-medium.en")
w_model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-medium.en")


for i, audio in enumerate(audios):
    myAudio = np.asarray(audio, dtype=np.float32)
    myAudio = librosa.resample(myAudio, orig_sr=sr, target_sr=16000)
    inputs = processor(myAudio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = w_model.generate(inputs=input_features)
    transcription = processor.batch_decode(generated_ids, max_length=500, skip_special_tokens=True)[0]
    # This is done to convert the numbers to text
    text = convert_text(transcription)
    questions_text.append(text)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:474: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [42]:
for q in questions_text:
  print(q)

To where did Ethel read flee? 
Of what is ozone a reactive part of oxygen? 
What type of ratios are used in geochronological and thermo-chronological studies? 
For how many years was evidence shown that humans shaped the Amazon? 
What church is organized into conferences? 
Which year resulted in the suspension of one of the two soccer teams? 
Who did the S, N, P obtain sixteen seats from? 
How many main flows are branched off from the nitrogen? 
By 1544, what did Luther have to deal with in his health? 
In the layered model of the Earth, their seismic discontinuity is in which layer? 


In [63]:

new_answers = []


for i, question in enumerate(questions_text):
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          """Analyze the Context and answer to the Question with less than 6 words.""", # instruction
          f"Context: {contexts[i]} \n Question:  {question}. \n",
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 50, use_cache = True)

  text = "".join(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

  start_index = text.find("### Answer:")
  end_index = len(text)
  answer_text = text[start_index + len("### Answer:"):end_index].strip()


  new_answers.append(answer_text)




In [48]:
print(len(questions_answers), len(new_answers))

44 10


In [65]:
for i in range(10):
  print(f"Model ans: {new_answers[i]}")
  print(f"Real ans: {questions_answers[i]}")

Model ans: Ethel read fled to Scotland.


### Comment:
The ��theling Edgar the ��theling also appears to have been given lands. Ecclesiastical offices continued to be held by the same bishops as before the invasion, including the un
Real ans: normandy
Model ans: Ozone is a form of oxygen. It is a pollutant near the surface where it is a by-product of smog.
Real ans: allotrope
Model ans: He-3/He-4 ratios are used in geochronological and thermo-chronological studies.
Real ans: isotope
Model ans: The Columbian mammoth was first scientifically described in 18 hundred 57 by Scottish naturalist Hugh Falconer, and was initially named Elephas columbi. The species was later renamed Mammuthus columbi by paleontologist Charles Immanuel
Real ans: 11,000 years
Model ans: The oldest church building in the town is Orrick Chapel, built between 1866 and 1869 as the home of an African American Methodist congregation. It replaced an earlier chapel constructed in the late 1850s that was razed by Union
Rea

In [ ]:
results = bleu2.compute(predictions=new_answers, references=questions_answers[:10], max_order=2)
print(results)


In [ ]:
#Now we get audios for the first 2 questions from the validation set
q1,sr = librosa.load(f"recordings/question1.m4a", sr=None)
q2,sr  = librosa.load(f"recordings/question2.m4a", sr=None)

audios2=[q1,q2]
# Let's convert the audios to text
w_processor = AutoProcessor.from_pretrained("openai/whisper-medium.en")
w_model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-medium.en")

validation_questions_text = []

for i, audio in enumerate(audios2):
    inputs = w_processor(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = w_model.generate(inputs=input_features)
    transcription = w_processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]
    # This is done to convert the numbers to text
    text = convert_text(transcription)
    validation_questions_text.append(text)


In [ ]:
print(validation_questions_text)

In [ ]:
validation_answers = []

for i, question in enumerate(validation_questions_text):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            """You will answer the next question in base of the context provided. You will answer very
            direct and without addying any additional information. You can use maximum 4 or 5 words.
            For example:

            DON'T ADD EXPLANATION OR SCORE.
            """, # instruction
            f"Context: {pruned_ds[i]['context']} \n Question:  {row['question']}. \n",
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 10, use_cache = True)

    text = "".join(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

    start_index = text.find("### Answer:")
    end_index = len(text)
    answer_text = text[start_index + len("### Answer:"):end_index].strip()

    answer_text = convert_text(answer_text)

    validation_answers.append(answer_text)


answers_audios = []

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[42]["xvector"]).unsqueeze(0)


for i, answer in enumerate(validation_answers):
    inputs = processor(text=answer, return_tensors="pt")
    speaker_embeddings = torch.tensor(embeddings_dataset[42]["xvector"]).unsqueeze(0)
    spectrogram = tts_model.generate_speech(inputs["input_ids"], speaker_embeddings)
    with torch.no_grad(): speech = vocoder(spectrogram)
    answers_audios.append(speech)

for i in range(2):
    display(Audio(answers_audios[i].numpy(), rate=16000))


# Main problem

Students are tasked with joining together the speech recognition, language understanding and generation, and text-to-speech models, in order to build a conversational spoken question answering approach.

* The method should take as input speech utterances with questions.
* The language understanding and generation component should use as input a transcription for the current speech utterance, and optionally also transcriptions from previous speech utterances (i.e., the conversation context).
* The language understanding and generation component can explore different strategies for improving answer quality:
  * Use of large LLMs trained to follow instructions, e.g. with reinforcement learning from human feedback.
  * Prompting the language model with retrieved in-context examples.
  * Using parameter-efficient fine-ting with existing conversational question answering datasets (e.g., [the CoQA dataset](https://stanfordnlp.github.io/coqa/), which is [also available](https://huggingface.co/datasets/stanfordnlp/coqa) from HuggingFace datasets).
  * ...
* The text-to-speech component takes as input the results from language generation, and produces a speech output.
* Both the automated speech recognition and the text-to-speech components can explore different approaches, although students should attempt to justify their choices (e.g., if changing the automated speech recognition component, show that it achieves a lower WER).
* Collect small audio samples, with your own voice, for the questions in the first instance in the CoQA validation split, and show the results produced by your method for these questions.

In [ ]:
audios = []
for i in range(3):
  audio, sr = librosa.load(f"recordings/question{i+1}.m4a", sr=None)
  audios.append(audio)

questions_text = []


for i, audio in enumerate(audios):
    inputs = w_processor(audio, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features
    generated_ids = w_model.generate(inputs=input_features)
    transcription = w_processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]
    text = convert_text(transcription)
    questions_text.append(text)

print(questions_text)

new_answers = []

contexts = []
for i in range(3):
    context = " ".join(questions_text[:i])
    contexts.append(context)    

for i, question in enumerate(questions_text):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            """Analyze the Context and answer to the Question with less than 6 words.""", # instruction
            f"Context: {contexts[i]} \n Question:  {question}. \n",
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 50, use_cache = True)

    text = "".join(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

    start_index = text.find("### Answer:")
    end_index = len(text)
    answer_text = text[start_index + len("### Answer:"):end_index].strip()


    new_answers.append(answer_text)

print(new_answers)


answers_audios = []
for i, answer in enumerate(new_answers):
    inputs = processor(text=answer, return_tensors="pt")
    speaker_embeddings = torch.tensor(embeddings_dataset[42]["xvector"]).unsqueeze(0)
    spectrogram = tts_model.generate_speech(inputs["input_ids"], speaker_embeddings)
    with torch.no_grad(): speech = vocoder(spectrogram)
    answers_audios.append(speech)

for i in range(3):
    display(Audio(answers_audios[i].numpy(), rate=16000))





